In [ ]:
""" 
    Parameters
        ----------
        img_path : file path to .tif file
            Location of image for inference 3-band (RGB) and 10cm spatial resolution
        out_path : folder path 
            Location where model output will be written 
        site : str
            Name to be given to model output 
  
        tile_size : int
            Value identifying desired forest stand size 
            E.G. TreeCRowNN accepts tiles with size 128, either use this or padding to achieve correct tile dimensions
        patch_size : int
            Value identifying the desired dimensions of extracted tiles
            E.G. A value of 0 should be used for tiles with dimension 128pix X 128pix
        padding : int
            Value identifying the desired amount of zero padding to be added to tiles to reach dims 128p X 128p
        batch_size : int
            Size of batch to be sent to model for inference
        minval : int
            minimum data value of input image
        maxval : int
            maximum data value of input image
            
        generate_heatmap : bool
            enter True to generate activation map in addition to FSD map
            if True, processing time per stand will increase ~4x
        normalize_heatmap : str
            enter "local", "global", or "none"
            if local, heatmap will be normalized per stand (useful to check for border issues)
            if global, heatmap will be normalized for full input image (useful to check for image trends)
            if none, no normalization will be applied
            
    Returns
        -------
        Array with predictions entered as integers, this array saved to .tif file in out_path location
        List of predictions
        Inference processing time in seconds
        
        Output will not be georeferenced, please use GDAL script to transfer metadata for use in a GIS
"""

## Set Params

In [1]:
img_path = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_ortho/2019_ort_4inf.tif'
#img_path = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_ortho/2019_ort_4inf_small2.tif'
#img_path = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_mine_site/MtP_Mine_clip_small.tif'
#img_path = r'F:/MtPolley/tree_counting/10cm_Resamples/2022_USherbrooke_QCNL/Site_20_10cm.tif'
#img_path = r'F:/MtPolley/tree_counting/10cm_Resamples/2022_UofC/StonyWest_10cm_clip.tif'
#img_path = r'F:/MtPolley/tree_counting/10cm_Resamples/2022_UofC/StonyFen_10cm.tif'
#img_path = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_ortho/2019_ort_clip_AI_wrkshp.tif'
out_path = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/'
#img_path = r'F:/MtPolley/tree_counting/10cm_Resamples/2021_SumacSouthON_10cm/n9_10cm.tif'

site = "newV1_2019ort_"
rgba_out = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/'

NoData = -9999 #enter NoData value of the incoming img
tile_size = 128 #enter desired tile size between 64 - 128, default 128
batch_size = 100 #enter batch size, max 250, default 100

minval = 0 #enter min val for normalization, default 0
maxval = 255 #enter max val for normalization, default 255

generate_heatmap = True #enter 'False' or 'True'
heatmap_algo = "cgc" #enter 'gc' for Grad-CAM, 'cgc' for custom Grad-CAM, or 'gcpp' for Grad-CAM++
target_layer1 = "batch_normalization_23" #"conv2d_23" #conv layer you want to view activations for
target_layer2 = "batch_normalization_13" #"batch_normalization_7","batch_normalization_19"]
           
normalize_heatmap = "local" #enter 'none', 'local', or 'global'
generate_rgba = False

k1=3
k2=7
k3=15

# Set Up

In [2]:
import os, sys, PIL,time,ipynb.fs, session_info,cv2
import numpy as np
import tifffile as tiff
from PIL import Image
Image.MAX_IMAGE_PIXELS = 5000000000

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.pyplot import imshow,figure
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
#from keras import models,layers
#from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
#from tensorflow.keras.models import Model, Sequential, load_model
#from tensorflow.keras.optimizers import Adam
print(tf.__version__)
print(f"Tensorflow ver. {tf.__version__}")
physical_device = tf.config.experimental.list_physical_devices('CPU')
print(f'Device found : {physical_device}')
#tf.config.experimental.get_memory_growth(physical_device[0])
K.clear_session()

#from ipynb.fs.full.TreeCRowNNv2_Functions_2 import *
session_info.show()

2.8.0
Tensorflow ver. 2.8.0
Device found : [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Import Data

In [11]:
padding = int((128-tile_size)/2)
print(f"padding : {padding}")

padding : 0


In [12]:
image = tiff.imread(img_path)
image[image==NoData] = 0 #np.nan
img = image[:,:,:3]
print(img.min(),img.max())

for i in range(0,2):
    crop_image(img[:,:,i],tile_size)

print(img.min(),img.max(),img.shape)

0 255
47 59951 39 36391
47 59951 39 36391
0 255 (36430, 59998, 3)


In [ ]:
plt.imshow(img/255)

In [13]:
y,x,chan = img.shape
big_x = round(int(x/tile_size),0)
cropx = (big_x*tile_size)
big_y = round(int(y/tile_size),0)
cropy = (big_y*tile_size)

In [14]:
blank_img = np.zeros([big_y,big_x],dtype=int)
blank_img2 = np.zeros([y,x],dtype=np.float32)
print(blank_img.shape,blank_img2.shape)

(284, 468) (36430, 59998)


In [15]:
if heatmap_algo == "cgc":
    blank = np.zeros([y,x],dtype=np.float32)
    blank_img3 = np.dstack([blank,blank,blank])

    print(blank_img3.shape)

(36430, 59998, 3)


## Load Model and Weights

In [16]:
NN_name='TreeCRowNNv2_V1_newdataWbblim_x1'
def tc_model(lr,spe,u,u2,u3,k1,k2,k3):
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    in1 = Input(shape=(128,128,3))
    
    conv1 = Conv2D(u,(k1,k1),activation='relu', padding='same')(in1)
    #conv1.trainable = False
    BN1 = BatchNormalization()(conv1)
    #BN1.trainable = False
    conv2 = Conv2D(u,(k1,k1),activation='relu', padding='same')(BN1)
    #conv2.trainable = False
    BN2 = BatchNormalization()(conv2)
    #BN2.trainable = False
 
    small1 = Conv2D(u,(k1,k1), activation='relu', padding='same')(BN2)
    #small1.trainable = False
    BN3 = BatchNormalization()(small1)
    #BN3.trainable = False
    small2 = Conv2D(u,(k1,k1), activation='relu', padding='same')(BN3)
    #small2.trainable = False
    BN4 = BatchNormalization()(small2)
    #BN4.trainable = False
    small3 = MaxPooling2D((2,2))(BN4)
    #small3.trainable = False
    small4 = Dropout(0.2)(small3)
    
    small5 = Conv2D(u,(k1,k1), activation='relu',padding='same')(small4)
    #small5.trainable = False
    BN5 = BatchNormalization()(small5)
    #BN5.trainable = False
    small6 = Conv2D(u,(k1,k1), activation='relu',padding='same')(BN5)
    #small6.trainable = False
    BN6 = BatchNormalization()(small6)
    #BN6.trainable = False
    small7 = Dropout(0.2)(BN6)
    
    small8 = Conv2D(u,(k1,k1), activation='relu',padding='same')(small7)
    #small8.trainable = False
    BN7 = BatchNormalization()(small8)
    #BN7.trainable = False
    small9 = Conv2D(u,(k1,k1), activation='relu',padding='same')(BN7)
    #small9.trainable = False
    BN8 = BatchNormalization()(small9)
    #BN8.trainable = False
    small10 = MaxPooling2D((2,2))(BN8)
    #small10.trainable = False
    small11 = Dropout(0.2)(small10)

    med1 = Conv2D(u,(k2,k2), activation='relu', padding='same')(BN2)
    #med1.trainable = False
    BN9 = BatchNormalization()(med1)
    #BN9.trainable = False
    med2 = Conv2D(u,(k2,k2), activation='relu', padding='same')(BN9)
    #med2.trainable = False
    BN10 = BatchNormalization()(med2)
    #BN10.trainable = False
    med3 = MaxPooling2D((2,2))(BN10)
    #med3.trainable = False
    med4 = Dropout(0.2)(med3)
    
    med5 = Conv2D(u,(k2,k2), activation='relu',padding='same')(med4)
    #med5.trainable = False
    BN11 = BatchNormalization()(med5)
    #BN11.trainable = False
    med6 = Conv2D(u,(k2,k2), activation='relu',padding='same')(BN11)
    #med6.trainable = False
    BN12 = BatchNormalization()(med6)
    #BN12.trainable = False
    med7 = Dropout(0.2)(BN12)
    
    med8 = Conv2D(u,(k2,k2), activation='relu',padding='same')(med7)
    #med8.trainable = False
    BN13 = BatchNormalization()(med8)
    #BN13.trainable = False
    med9 = Conv2D(u,(k2,k2), activation='relu',padding='same')(BN13)
    #med9.trainable = False
    BN14 = BatchNormalization()(med9)
    #BN14.trainable = False
    med10 = MaxPooling2D((2,2))(BN14)
    #med10.trainable = False
    med11 = Dropout(0.2)(med10)

    big1 = Conv2D(u,(k3,k3), activation='relu', padding='same')(BN2)
    #big1.trainable = False
    BN15 = BatchNormalization()(big1)
    #BN15.trainable = False
    big2 = Conv2D(u,(k3,k3), activation='relu', padding='same')(BN15)
    #big2.trainable = False
    BN16 = BatchNormalization()(big2)
    #BN16.trainable = False
    big3 = MaxPooling2D((2,2))(BN16)
    #big3.trainable = False
    big4 = Dropout(0.2)(big3)
    
    big5 = Conv2D(u,(k3,k3), activation='relu',padding='same')(big4)
    #big5.trainable = False
    BN17 = BatchNormalization()(big5)
    #BN17.trainable = False
    big6 = Conv2D(u,(k3,k3), activation='relu',padding='same')(BN17)
    #big6.trainable = False
    BN18 = BatchNormalization()(big6)
    #BN18.trainable = False
    big7 = Dropout(0.2)(BN18)
    
    big8 = Conv2D(u,(k3,k3), activation='relu',padding='same')(big7)
    #big8.trainable = False
    BN19 = BatchNormalization()(big8)
    #BN19.trainable = False
    big9 = Conv2D(u,(k3,k3), activation='relu',padding='same')(BN19)
    #big9.trainable = False
    BN20 = BatchNormalization()(big9)
    #BN20.trainable = False
    big10 = MaxPooling2D((2,2))(BN20)
    #big10.trainable = False
    big11 = Dropout(0.2)(big10)

    concat1 = tf.keras.layers.Concatenate()([small11,med11,big11])

    FC1 = Conv2D(u2,(1,1), activation='relu',padding='valid')(concat1)
    BN21 = BatchNormalization()(FC1)
    FC2 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN21)
    BN22 = BatchNormalization()(FC2)
    FC3 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN22)
    BN23 = BatchNormalization()(FC3)
    FC4 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN23)
    BN24 = BatchNormalization()(FC4)

    
    flat = Flatten()(BN24)

    dense1 = keras.layers.Dense(u3, activation='relu')(flat)
    BN25 = BatchNormalization()(dense1)
    dense2 = keras.layers.Dense(u3, activation='relu')(BN25)
    BN26 = BatchNormalization()(dense2)
    dense3 = keras.layers.Dense(u3, activation='relu')(BN26)
    BN27 = BatchNormalization()(dense3)
    dense4 = keras.layers.Dense(u3, activation='relu')(BN27)
    BN28 = BatchNormalization()(dense4)
    dense5 = keras.layers.Dense(u3, activation='relu')(BN28)
    BN29 = BatchNormalization()(dense5)
    out1 = keras.layers.Dense(1, activation='linear')(BN29)

    model = Model(inputs=[in1], outputs=[out1])

    model.compile(loss="MeanAbsoluteError", 
              optimizer =keras.optimizers.Adam(learning_rate=lr),
              metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError()])

    return model


In [17]:
#load best model and evaluate
#lr,spe,u,u2,u3 = 0.001,160,16,16,16
#lr,spe,u,u2,u3 = 0.001,160,8,16,12
lr,spe,u,u2,u3,k1,k2,k3 = 0.001,160,16,12,12,3,7,15
#path_to_weights = r"C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/log/logsGRSimple_xtra0_+dense/GRSimple_xtra0_+dense-2.11927mae-0r-[160, 0.001, 16, 16, 16]p-14297.24sec/Weights-2.11927_03142023_232824.h5"
#path_to_weights = r'C:/Users/jlovitt/Pyworking/padding/log/logsTreeCRowNNv2_wPaddingCombo_newdatasetV3c/TreeCRowNNv2_wPaddingCombo_newdatasetV3c-1.76054mae-16r-[160, 0.001, 12, 8, 8]p-6039.90sec/Weights-1.76054_08282023_054930.h5'
#path_to_weights = r'C:/Users/jlovitt/Pyworking/TreeCRowNN_Transfer/ALL/log/logsTreeCRowNNv2_V3c_newdataWbblim_x1/TreeCRowNNv2_V3c_newdataWbblim_x1-2.77685mae-13r-[160, 0.001, 8, 16, 12]p-4951.83sec/Weights-2.77685_09072023_021159.h5'
path_to_weights = r'C:/Users/jlovitt/Pyworking/TreeCRowNN_Transfer/ALL/log/logsTreeCRowNNv2_V1_newdataWbblim_x1/TreeCRowNNv2_V1_newdataWbblim_x1-2.67247mae-10r-[160, 0.001, 16, 12, 12]p-7053.12sec/Weights-2.67247_09142023_043256.h5'

#path_to_weights = r"C:/Users/jlovitt/Pyworking/padding/log/logsTreeCRowNNv2_wPaddingCombo_refinedV1/TreeCRowNNv2_wPaddingCombo_refinedV1-1.56268mae-32r-[160, 0.001, 8, 18, 8]p-17184.02sec/Weights-1.56268_07152023_231140.h5"
#path_to_weights = r"C:/Users/jlovitt/Pyworking/padding/log/logsTreeCRowNNv2_wPaddingCombo_refinedV2/TreeCRowNNv2_wPaddingCombo_refinedV2-1.69208mae-0r-[160, 0.001, 16, 16, 16]p-8490.73sec/Weights-1.69208_07282023_222908.h5"
#path_to_weights = r"C:/Users/jlovitt/Pyworking/padding/log/logsTreeCRowNNv2_wPaddingCombo_newdatasetV3/TreeCRowNNv2_wPaddingCombo_newdatasetV3-1.71451mae-31r-[160, 0.001, 12, 16, 16]p-8028.14sec/Weights-1.71451_08092023_103242.h5"
#path_to_weights = r"C:/Users/jlovitt/Pyworking/padding/log/logsTreeCRowNNv2_wPaddingCombo_newdatasetV1b/TreeCRowNNv2_wPaddingCombo_newdatasetV1b-1.71872mae-0r-[160, 0.001, 8, 14, 16]p-7364.81sec/Weights-1.71872_08162023_170218.h5"
#path_to_weights = r"C:/Users/jlovitt/Pyworking/padding/log/logsTreeCRowNNv2_wPaddingCombo_newdatasetV1c/TreeCRowNNv2_wPaddingCombo_newdatasetV1c-1.70292mae-35r-[160, 0.001, 14, 8, 16]p-4857.87sec/Weights-1.70292_08242023_022059.h5"

model=tc_model(lr,spe,u,u2,u3,k1,k2,k3)
model.load_weights(path_to_weights)
#

## Perform Batch Inference

## Settings
site = "heatmap_test_"
rgba_out = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/heat_test/'

NoData = -9999 #enter NoData value of the incoming img
tile_size = 128 #enter desired tile size between 64 - 128, default 128
batch_size = 100 #enter batch size, max 250, default 100

minval = 0 #enter min val for normalization, default 0
maxval = 255 #enter max val for normalization, default 255

generate_heatmap = True #enter 'False' or 'True'
heatmap_algo = "gc" #enter 'gc' for Grad-CAM or 'gcpp' for Grad-CAM++
target_layer = "batch_normalization_23" #"conv2d_23" #conv layer you want to view activations for
normalize_heatmap = "none" #enter 'none', 'local', or 'global'
generate_rgba = False

k1=3
k2=16
k3=64

In [ ]:
#tree_counts2,T2 = create_image2(img,blank_img,tile_size,model,chan,padding,minval,maxval,batch_size)
T2 = create_image2(img,
                   blank_img,
                   blank_img2,
                   tile_size,
                   model,
                   minval,
                   maxval,
                   batch_size,
                   #"conv2d_23",
                   "batch_normalization_23",
                   "batch_normalization_13",
                   generate_heatmap,
                   heatmap_algo,
                   normalize_heatmap,
                   generate_rgba,
                   rgba_out)
print((big_y*big_x)," BLOCKS PROCESSED IN ",round(T2,3), ' SECONDS')
print("TOTAL TIME PER BLOCK : ", (T2/(big_y*big_x)),"sec")

TOTAL BLOCKS :  132912
BATCH SIZE :  100


In [ ]:
total_trees = str(np.ndarray.sum(blank_img))
m = tile_size/10
area = round(((big_y*big_x)*(m*m)),2)
print("TOTAL TREES PREDICTED WITH TILE SIZE " +str(tile_size)+" : " + total_trees)  
print("(COL, ROW)",blank_img.shape)
print("TOTAL AREA PROCESSED : ",area, "m2 --> ",round(area/10000,2), "ha")
print("TOTAL PROCESSING TIME : ",round(T2/60,1),"MINUTES")
print("PREDICTED RANGE : ",blank_img.min()," to ",blank_img.max(), " TREES")

#plt.subplot(1,2,1)
#plt.set_title("INPUT")
#plt.imshow(img//255)
#plt.subplot(1,2,2)
#plt.set_title("PREDICTION")
plt.imshow(blank_img)
#plt.tight_layout()
#plt.show()

In [ ]:
total_trees = str(np.ndarray.sum(blank_img))
plt.imshow(blank_img2)
print("TOTAL TREES PREDICTED WITH TILE SIZE " +str(tile_size)+" : " + total_trees)  
print(blank_img.shape)
blank_img.min(),blank_img.max()

## Write Output

In [ ]:
#write FSD map
tiff.imwrite(str(os.path.join(out_path+site+"_totaltrees_"+total_trees+'.tif')),blank_img)

In [ ]:
normalize_heatmap='global'

In [ ]:
#write activation map if requested
if generate_heatmap == True:
    if normalize_heatmap == "local":
        tiff.imwrite(str(os.path.join(out_path+site+"ts"+str(tile_size)+"_heatmap_localNorm_"+str(heatmap_algo)+'.tif')),blank_img2)
    elif normalize_heatmap == "global":
        n_heatmap = normalize_activations(blank_img3)
        tiff.imwrite(str(os.path.join(out_path+site+"ts"+str(tile_size)+"_heatmap_globalNorm_"+str(heatmap_algo)+'.tif')),n_heatmap)
    elif normalize_heatmap == "none":
        tiff.imwrite(str(os.path.join(out_path+site+"ts"+str(tile_size)+"heatmap_noNorm_"+str(heatmap_algo)+'.tif')),blank_img2) 

## activation map test

In [ ]:
def get_predict(patch):
    import numpy as np
    prediction = np.mean(patch)
    prediction = np.rint(prediction)
    if prediction < 0.0:
        prediction = 0.0 
    return prediction

def create_image2(img,blank_img,blank_img2,tile_size,model,chan,padding,minval,maxval,batch_size,target_layer):
    import numpy as np
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    t1 = time.time()
    predictions = []
    activations = []
        
    y,x,z = img.shape
    nblocks_row = int(y//tile_size)
    nblocks_col = int(x//tile_size)
    total_blocks = nblocks_row*nblocks_col
    print("TOTAL BLOCKS : ",total_blocks)
    print("BATCH SIZE : ",batch_size)
        
    row_list = range(0,nblocks_row)
    col_list = range(0,nblocks_col)
        
    n = 0
    dataset = []
    coords = []
    blocks = []
        
    for i in row_list:
        for j in col_list:
                
            row_start = i
            row_start = row_start*tile_size
            row_end = row_start + tile_size
            col_start = j
            col_start = col_start*tile_size
            col_end = col_start + tile_size
            
            block = np.array([row_start,row_end,col_start,col_end]).astype('int32')
            blocks.append(block)
        
            patch = img[row_start:row_end,col_start:col_end]
            patch = tf.cast(patch,tf.float32)/maxval
            patch = tf.reshape(patch, shape=[1, tile_size, tile_size, chan])
            patch = tf.keras.layers.ZeroPadding2D(padding=padding)(patch)
            coords.append([i,j])
            dataset.append(patch)
            n+=1
            if len(dataset)>=batch_size:
                dataset2 = tf.data.Dataset.from_tensor_slices(dataset)
                result = model.predict(dataset2)
                for k in range (len(result)):
                    predict = get_predict(result[k])
                    blank_img[coords[k][0],coords[k][1]]=predict
                    predictions.append(result[k])
                coords=[]
                dataset = []
            if str(n)[-4:]=='0000':
                t2 = time.time()-t1
                print('{} pixels processed in {}, {}% done'.format(n,t2,str("%.2f"%(n/total_blocks*100))))
    print("FINAL PIXELS")
    dataset2 = tf.data.Dataset.from_tensor_slices(dataset)
    result = model.predict(dataset2)
    for k in range (len(result)):
        predict = get_predict(result[k])
        blank_img[coords[k][0],coords[k][1]]=predict
        predictions.append(result[k])
    
    t3=time.time()     
    total_time = t3-t1
        
    return predictions,blocks,total_time

In [ ]:
def make_heatmap(img_array, model, last_conv, tile_size, padding, pred_index=None):
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    import cv2
    grad_model = keras.models.Model(model.inputs,[model.get_layer(last_conv).output,model.output])
    
    with tf.GradientTape() as tape:
        last_conv_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:,pred_index]
        
        grads = tape.gradient(class_channel, last_conv_output)
        pooled_grads = tf.reduce_mean(grads,axis=(0,1,2))
        
        last_conv_output = last_conv_output[0]
        heatmap = last_conv_output @ pooled_grads[...,tf.newaxis]
        heatmap = tf.squeeze(heatmap)         
        #heatmap = tf.maximum(heatmap,-1)/tf.math.reduce_max(heatmap)
        #heatmap = heatmap/tf.math.reduce_max(heatmap)
        #print(heatmap.numpy())
        heatmap = cv2.resize(heatmap.numpy(),(tile_size+(2*padding),tile_size+(2*padding)))
        #heatmap = (heatmap*255).astype("uint8")
        #print(heatmap)
        
        if padding != 0:
            heatmap = heatmap[padding:-padding,padding:-padding]
        return heatmap

In [ ]:
def get_heatmap(in1, model, last_conv,tile_size,padding,pred_index=None):
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras
    import cv2
    
    grad_model = keras.models.Model(model.inputs,[model.get_layer(last_conv).output,model.output])
    
    with tf.GradientTape(persistent=False) as tape:
        last_conv_output, preds = grad_model(in1)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
            
        #generate activation heatmap
        class_channel = preds[:,pred_index]
        grads = tape.gradient(class_channel, last_conv_output)
        pooled_grads = tf.reduce_mean(grads,axis=(0,1,2))
            
        last_conv_output = last_conv_output[0]
        heatmap = last_conv_output @ pooled_grads[...,tf.newaxis]
        heatmap = tf.squeeze(heatmap)
            
        heatmap = cv2.resize(heatmap.numpy(),(tile_size+(2*padding),tile_size+(2*padding)))
        heatmap = heatmap[padding:-padding,padding:-padding]
            
        numer = heatmap-np.min(heatmap)
        denom = (heatmap.max()-heatmap.min())+1e-8
        heatmap = numer/denom
        #heatmap = (heatmap*255).astype("uint8") 
    
    return heatmap
                                 
    

In [ ]:
def get_predict(patch):
    import numpy as np
    prediction = np.mean(patch)
    prediction = np.rint(prediction)
    if prediction < 0.0:
        prediction = 0.0 
    return prediction

def create_image2(img,blank_img,blank_img2,tile_size,model,chan,padding,minval,maxval,batch_size,target_layer):
    import numpy as np
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    t1 = time.time()
    predictions = []
    activations = []
        
    y,x,z = img.shape
    nblocks_row = int(y//tile_size)
    nblocks_col = int(x//tile_size)
    total_blocks = nblocks_row*nblocks_col
    print("TOTAL BLOCKS : ",total_blocks)
    print("BATCH SIZE : ",batch_size)
        
    row_list = range(0,nblocks_row)
    col_list = range(0,nblocks_col)
        
    n = 0
    dataset = []
    coords = []
    blocks = []
        
    for i in row_list:
        for j in col_list:
                
            row_start = i
            row_start = row_start*tile_size
            row_end = row_start + tile_size
            col_start = j
            col_start = col_start*tile_size
            col_end = col_start + tile_size
            
            block = np.array([row_start,row_end,col_start,col_end]).astype('int32')
            blocks.append(block)
        
            patch = img[row_start:row_end,col_start:col_end]
            patch = tf.cast(patch,tf.float32)/maxval
            patch = tf.reshape(patch, shape=[1, tile_size, tile_size, chan])
            patch = tf.keras.layers.ZeroPadding2D(padding=padding)(patch)
            
            heatmap = get_heatmap(patch,model, target_layer,tile_size,padding)
            blank_img2[block[0]:block[1],block[2]:block[3]]=heatmap
            
            coords.append([i,j])
            dataset.append(patch)
            n+=1
            if len(dataset)>=batch_size:
                dataset2 = tf.data.Dataset.from_tensor_slices(dataset)
                result = model.predict(dataset2)
                for k in range (len(result)):
                    predict = get_predict(result[k])
                    blank_img[coords[k][0],coords[k][1]]=predict
                    predictions.append(result[k])
                coords=[]
                dataset = []
            if str(n)[-4:]=='0000':
                t2 = time.time()-t1
                print('{} pixels processed in {}, {}% done'.format(n,t2,str("%.2f"%(n/total_blocks*100))))
    print("FINAL PIXELS")
    dataset2 = tf.data.Dataset.from_tensor_slices(dataset)
    result = model.predict(dataset2)
    for k in range (len(result)):
        predict = get_predict(result[k])
        blank_img[coords[k][0],coords[k][1]]=predict
        predictions.append(result[k])
    
    t3=time.time()     
    total_time = t3-t1
        
    return predictions,blocks,total_time

In [ ]:
tree_counts2,img_blocks,T2 = create_image2(img,blank_img,blank_img2,tile_size,model,chan,padding,minval,maxval,batch_size,"batch_normalization_23")

In [ ]:
plt.imshow(blank_img2)

In [ ]:
print((big_y*big_x)," BLOCKS PROCESSED IN ",round(T2,3), ' SECONDS')
print("TOTAL TIME PER BLOCK : ", (T2/(big_y*big_x)),"sec")

In [ ]:
tiff.imwrite(str(os.path.join(out_path+site+'_activations'+'.tif')),blank_img2)

NN_name='TreeCRowNNv2_wPaddingCombo_newdatasetV1c'
def tc_model(lr,spe,u,u2,u3):
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    in1 = Input(shape=(128,128,3))
    
    conv1 = Conv2D(u,(3,3),activation='relu', padding='same')(in1)
    BN1 = BatchNormalization()(conv1)
    conv2 = Conv2D(u,(3,3),activation='relu', padding='same')(BN1)
    BN2 = BatchNormalization()(conv2)
 
    small1 = Conv2D(u,(3, 3), activation='relu', padding='same')(BN2)
    BN3 = BatchNormalization()(small1)
    small2 = Conv2D(u,(3, 3), activation='relu', padding='same')(BN3)
    BN4 = BatchNormalization()(small2)
    small3 = MaxPooling2D((2,2))(BN4)
    small4 = Dropout(0.2)(small3)
    
    small5 = Conv2D(u,(3,3), activation='relu',padding='same')(small4)
    BN5 = BatchNormalization()(small5)
    small6 = Conv2D(u,(3,3), activation='relu',padding='same')(BN5)
    BN6 = BatchNormalization()(small6)
    small7 = Dropout(0.2)(BN6)
    
    small8 = Conv2D(u,(3,3), activation='relu',padding='same')(small7)
    BN7 = BatchNormalization()(small8)
    small9 = Conv2D(u,(3,3), activation='relu',padding='same')(BN7)
    BN8 = BatchNormalization()(small9)
    small10 = MaxPooling2D((2,2))(BN8)
    small11 = Dropout(0.2)(small10)

    med1 = Conv2D(u,(7, 7), activation='relu', padding='same')(BN2)
    BN9 = BatchNormalization()(med1)
    med2 = Conv2D(u,(7, 7), activation='relu', padding='same')(BN9)
    BN10 = BatchNormalization()(med2)
    med3 = MaxPooling2D((2,2))(BN10)
    med4 = Dropout(0.2)(med3)
    
    med5 = Conv2D(u,(7, 7), activation='relu',padding='same')(med4)
    BN11 = BatchNormalization()(med5)
    med6 = Conv2D(u,(7, 7), activation='relu',padding='same')(BN11)
    BN12 = BatchNormalization()(med6)
    med7 = Dropout(0.2)(BN12)
    
    med8 = Conv2D(u,(7, 7), activation='relu',padding='same')(med7)
    BN13 = BatchNormalization()(med8)
    med9 = Conv2D(u,(7, 7), activation='relu',padding='same')(BN13)
    BN14 = BatchNormalization()(med9)
    med10 = MaxPooling2D((2,2))(BN14)
    med11 = Dropout(0.2)(med10)

    big1 = Conv2D(u,(15, 15), activation='relu', padding='same')(BN2)
    BN15 = BatchNormalization()(big1)
    big2 = Conv2D(u,(15, 15), activation='relu', padding='same')(BN15)
    BN16 = BatchNormalization()(big2)
    big3 = MaxPooling2D((2,2))(BN16)
    big4 = Dropout(0.2)(big3)
    
    big5 = Conv2D(u,(15, 15), activation='relu',padding='same')(big4)
    BN17 = BatchNormalization()(big5)
    big6 = Conv2D(u,(15, 15), activation='relu',padding='same')(BN17)
    BN18 = BatchNormalization()(big6)
    big7 = Dropout(0.2)(BN18)
    
    big8 = Conv2D(u,(15, 15), activation='relu',padding='same')(big7)
    BN19 = BatchNormalization()(big8)
    big9 = Conv2D(u,(15, 15), activation='relu',padding='same')(BN19)
    BN20 = BatchNormalization()(big9)
    big10 = MaxPooling2D((2,2))(BN20)
    big11 = Dropout(0.2)(big10)

    concat1 = tf.keras.layers.Concatenate()([small11,med11,big11])

    FC1 = Conv2D(u2,(1,1), activation='relu',padding='valid')(concat1)
    BN21 = BatchNormalization()(FC1)
    FC2 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN21)
    BN22 = BatchNormalization()(FC2)
    FC3 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN22)
    BN23 = BatchNormalization()(FC3)
    FC4 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN23)
    BN24 = BatchNormalization()(FC4)
    
    flat = Flatten()(BN24)

    dense1 = keras.layers.Dense(u3, activation='relu')(flat)
    BN25 = BatchNormalization()(dense1)
    dense2 = keras.layers.Dense(u3, activation='relu')(BN25)
    BN26 = BatchNormalization()(dense2)
    dense3 = keras.layers.Dense(u3, activation='relu')(BN26)
    BN27 = BatchNormalization()(dense3)
    dense4 = keras.layers.Dense(u3, activation='relu')(BN27)
    BN28 = BatchNormalization()(dense4)
    dense5 = keras.layers.Dense(u3, activation='relu')(BN28)
    BN29 = BatchNormalization()(dense5)
    out1 = keras.layers.Dense(1, activation='linear')(BN29)

    model = Model(inputs=[in1], outputs=[out1])

    model.compile(loss="MeanAbsoluteError", 
              optimizer =keras.optimizers.Adam(learning_rate=lr),
              metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError()])

    return model


NN_name='TreeCRowNNv2_wPaddingCombo_newdatasetV1'
def tc_model(lr,spe,u,u2,u3):
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    in1 = Input(shape=(128,128,3))
    
    conv1 = Conv2D(u,(3,3),activation='relu', padding='same')(in1)
    BN1 = BatchNormalization()(conv1)
    conv2 = Conv2D(u,(3,3),activation='relu', padding='same')(BN1)
    BN2 = BatchNormalization()(conv2)
 
    small1 = Conv2D(u,(3, 3), activation='relu', padding='same')(BN2)
    BN3 = BatchNormalization()(small1)
    small2 = Conv2D(u,(3, 3), activation='relu', padding='same')(BN3)
    BN4 = BatchNormalization()(small2)
    small3 = MaxPooling2D((2,2))(BN4)
    small4 = Dropout(0.2)(small3)
    
    small5 = Conv2D(u,(3,3), activation='relu',padding='same')(small4)
    BN5 = BatchNormalization()(small5)
    small6 = Conv2D(u,(3,3), activation='relu',padding='same')(BN5)
    BN6 = BatchNormalization()(small6)
    small7 = Dropout(0.2)(BN6)
    
    small8 = Conv2D(u,(3,3), activation='relu',padding='same')(small7)
    BN7 = BatchNormalization()(small8)
    small9 = Conv2D(u,(3,3), activation='relu',padding='same')(BN7)
    BN8 = BatchNormalization()(small9)
    small10 = MaxPooling2D((2,2))(BN8)
    small11 = Dropout(0.2)(small10)

    med1 = Conv2D(u,(7, 7), activation='relu', padding='same')(BN2)
    BN9 = BatchNormalization()(med1)
    med2 = Conv2D(u,(7, 7), activation='relu', padding='same')(BN9)
    BN10 = BatchNormalization()(med2)
    med3 = MaxPooling2D((2,2))(BN10)
    med4 = Dropout(0.2)(med3)
    
    med5 = Conv2D(u,(7, 7), activation='relu',padding='same')(med4)
    BN11 = BatchNormalization()(med5)
    med6 = Conv2D(u,(7, 7), activation='relu',padding='same')(BN11)
    BN12 = BatchNormalization()(med6)
    med7 = Dropout(0.2)(BN12)
    
    med8 = Conv2D(u,(7, 7), activation='relu',padding='same')(med7)
    BN13 = BatchNormalization()(med8)
    med9 = Conv2D(u,(7, 7), activation='relu',padding='same')(BN13)
    BN14 = BatchNormalization()(med9)
    med10 = MaxPooling2D((2,2))(BN14)
    med11 = Dropout(0.2)(med10)

    big1 = Conv2D(u,(15, 15), activation='relu', padding='same')(BN2)
    BN15 = BatchNormalization()(big1)
    big2 = Conv2D(u,(15, 15), activation='relu', padding='same')(BN15)
    BN16 = BatchNormalization()(big2)
    big3 = MaxPooling2D((2,2))(BN16)
    big4 = Dropout(0.2)(big3)
    
    big5 = Conv2D(u,(15, 15), activation='relu',padding='same')(big4)
    BN17 = BatchNormalization()(big5)
    big6 = Conv2D(u,(15, 15), activation='relu',padding='same')(BN17)
    BN18 = BatchNormalization()(big6)
    big7 = Dropout(0.2)(BN18)
    
    big8 = Conv2D(u,(15, 15), activation='relu',padding='same')(big7)
    BN19 = BatchNormalization()(big8)
    big9 = Conv2D(u,(15, 15), activation='relu',padding='same')(BN19)
    BN20 = BatchNormalization()(big9)
    big10 = MaxPooling2D((2,2))(BN20)
    big11 = Dropout(0.2)(big10)

    concat1 = tf.keras.layers.Concatenate()([small11,med11,big11])

    FC1 = Conv2D(u2,(1,1), activation='relu',padding='valid')(concat1)
    BN21 = BatchNormalization()(FC1)
    FC2 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN21)
    BN22 = BatchNormalization()(FC2)
    FC3 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN22)
    BN23 = BatchNormalization()(FC3)
    FC4 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN23)
    BN24 = BatchNormalization()(FC4)

    
    flat = Flatten()(BN24)

    dense1 = keras.layers.Dense(u3, activation='relu')(flat)
    BN25 = BatchNormalization()(dense1)
    dense2 = keras.layers.Dense(u3, activation='relu')(BN25)
    BN26 = BatchNormalization()(dense2)
    dense3 = keras.layers.Dense(u3, activation='relu')(BN26)
    BN27 = BatchNormalization()(dense3)
    dense4 = keras.layers.Dense(u3, activation='relu')(BN27)
    BN28 = BatchNormalization()(dense4)
    dense5 = keras.layers.Dense(u3, activation='relu')(BN28)
    BN29 = BatchNormalization()(dense5)
    out1 = keras.layers.Dense(1, activation='linear')(BN29)

    model = Model(inputs=[in1], outputs=[out1])

    model.compile(loss="MeanAbsoluteError", 
              optimizer =keras.optimizers.Adam(learning_rate=lr),
              metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError()])

    return model



NN_name='TreeCRowNNv2_wPaddingCombo_newdatasetV3c'
def tc_model(lr,spe,u,u2,u3):
    import time
    import tensorflow as tf
    from tensorflow import keras
    from keras import backend as K
    from keras import models,layers
    from tensorflow.keras.layers import Input, Conv2D,Conv1D, UpSampling2D,GlobalMaxPool2D,GlobalAveragePooling2D, concatenate,Dense, Flatten, Dropout,BatchNormalization, MaxPooling2D
    from tensorflow.keras.models import Model, Sequential, load_model
    from tensorflow.keras.optimizers import Adam
    
    in1 = Input(shape=(128,128,3))
    
    conv1 = Conv2D(u,(3,3),activation='relu', padding='same')(in1)
    BN1 = BatchNormalization()(conv1)
    conv2 = Conv2D(u,(3,3),activation='relu', padding='same')(BN1)
    BN2 = BatchNormalization()(conv2)
 
    small1 = Conv2D(u,(3, 3), activation='relu', padding='same')(BN2)
    BN3 = BatchNormalization()(small1)
    small2 = Conv2D(u,(3, 3), activation='relu', padding='same')(BN3)
    BN4 = BatchNormalization()(small2)
    small3 = MaxPooling2D((2,2))(BN4)
    small4 = Dropout(0.2)(small3)
    
    small5 = Conv2D(u,(3,3), activation='relu',padding='same')(small4)
    BN5 = BatchNormalization()(small5)
    small6 = Conv2D(u,(3,3), activation='relu',padding='same')(BN5)
    BN6 = BatchNormalization()(small6)
    small7 = Dropout(0.2)(BN6)
    
    small8 = Conv2D(u,(3,3), activation='relu',padding='same')(small7)
    BN7 = BatchNormalization()(small8)
    small9 = Conv2D(u,(3,3), activation='relu',padding='same')(BN7)
    BN8 = BatchNormalization()(small9)
    small10 = MaxPooling2D((2,2))(BN8)
    small11 = Dropout(0.2)(small10)

    med1 = Conv2D(u,(16, 16), activation='relu', padding='same')(BN2)
    BN9 = BatchNormalization()(med1)
    med2 = Conv2D(u,(16, 16), activation='relu', padding='same')(BN9)
    BN10 = BatchNormalization()(med2)
    med3 = MaxPooling2D((2,2))(BN10)
    med4 = Dropout(0.2)(med3)
    
    med5 = Conv2D(u,(16, 16), activation='relu',padding='same')(med4)
    BN11 = BatchNormalization()(med5)
    med6 = Conv2D(u,(16, 16), activation='relu',padding='same')(BN11)
    BN12 = BatchNormalization()(med6)
    med7 = Dropout(0.2)(BN12)
    
    med8 = Conv2D(u,(16, 16), activation='relu',padding='same')(med7)
    BN13 = BatchNormalization()(med8)
    med9 = Conv2D(u,(16, 16), activation='relu',padding='same')(BN13)
    BN14 = BatchNormalization()(med9)
    med10 = MaxPooling2D((2,2))(BN14)
    med11 = Dropout(0.2)(med10)

    big1 = Conv2D(u,(64, 64), activation='relu', padding='same')(BN2)
    BN15 = BatchNormalization()(big1)
    big2 = Conv2D(u,(64, 64), activation='relu', padding='same')(BN15)
    BN16 = BatchNormalization()(big2)
    big3 = MaxPooling2D((2,2))(BN16)
    big4 = Dropout(0.2)(big3)
    
    big5 = Conv2D(u,(64, 64), activation='relu',padding='same')(big4)
    BN17 = BatchNormalization()(big5)
    big6 = Conv2D(u,(64, 64), activation='relu',padding='same')(BN17)
    BN18 = BatchNormalization()(big6)
    big7 = Dropout(0.2)(BN18)
    
    big8 = Conv2D(u,(64, 64), activation='relu',padding='same')(big7)
    BN19 = BatchNormalization()(big8)
    big9 = Conv2D(u,(64, 64), activation='relu',padding='same')(BN19)
    BN20 = BatchNormalization()(big9)
    big10 = MaxPooling2D((2,2))(BN20)
    big11 = Dropout(0.2)(big10)

    concat1 = tf.keras.layers.Concatenate()([small11,med11,big11])

    FC1 = Conv2D(u2,(1,1), activation='relu',padding='valid')(concat1)
    BN21 = BatchNormalization()(FC1)
    FC2 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN21)
    BN22 = BatchNormalization()(FC2)
    FC3 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN22)
    BN23 = BatchNormalization()(FC3)
    FC4 = Conv2D(u,(5,5), activation='relu',padding='valid')(BN23)
    BN24 = BatchNormalization()(FC4)

    
    flat = Flatten()(BN24)

    dense1 = keras.layers.Dense(u3, activation='relu')(flat)
    BN25 = BatchNormalization()(dense1)
    dense2 = keras.layers.Dense(u3, activation='relu')(BN25)
    BN26 = BatchNormalization()(dense2)
    dense3 = keras.layers.Dense(u3, activation='relu')(BN26)
    BN27 = BatchNormalization()(dense3)
    dense4 = keras.layers.Dense(u3, activation='relu')(BN27)
    BN28 = BatchNormalization()(dense4)
    dense5 = keras.layers.Dense(u3, activation='relu')(BN28)
    BN29 = BatchNormalization()(dense5)
    out1 = keras.layers.Dense(1, activation='linear')(BN29)

    model = Model(inputs=[in1], outputs=[out1])

    model.compile(loss="MeanAbsoluteError", 
              optimizer =keras.optimizers.Adam(learning_rate=lr),
              metrics=[tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError()])

    return model
